In [1]:
import torch
from torch import nn,optim

In [20]:
class logistic_regression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(logistic_regression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        yhat = self.linear(x)
        return yhat
    
    def predict(self, X):
        lin = self.forward(X)
        # softmaxFn = nn.Softmax(dim=1)
        # output = softmaxFn(input)
        _, pos = torch.max(lin, dim=1) # returning the position of the max value which will be the class
        return pos
    
    def accuracy(self, x_test, y_test):
        y_pred = self.predict(x_test)
        return torch.mean((y_pred == y_test.type(torch.ByteTensor)).type(torch.float))

In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [25]:
# Load Iris dataset
iris = load_iris()
_x = iris.data
_y = iris.target

# stacking the new data together to use it for shuffling later
data = np.column_stack((_x, _y))
shufled_data = shuffle(data)

# new data after shuffling
x = shufled_data[:, 0:4]
y = shufled_data[:, 4]

# convert to tensors
X = torch.from_numpy(x.astype(np.float32))
Y = torch.from_numpy(y.astype(np.float32))

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [26]:
print(X_train.shape)
n_samples, n_features = X_train.shape

# setting up the model 
model = logistic_regression(n_features, 3)
optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()

torch.Size([105, 4])


In [29]:
num_epochs = 1000
def train_model(epochs, X, Y):    
    for epoch in range(epochs):
        yhat = model(X)
        loss = criterion(yhat, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')
        
Y_train = Y_train.long()
train_model(num_epochs, X_train, Y_train)

# print(type(X_test))
with torch.no_grad():
    predictions = model.predict(X_test)
    print("predictions: ", predictions)
    
    # Calculate accuracy 
    # Y_test = torch.argmax(Y_train, dim=1)
    print("Y_test: ", Y_test)
    acc = model.accuracy(X_test, Y_test)  
    print(f'accuracy: {acc.item():.4f}')

Epoch [0/1000], Loss: 0.1378
Epoch [100/1000], Loss: 0.1326
Epoch [200/1000], Loss: 0.1281
Epoch [300/1000], Loss: 0.1243
Epoch [400/1000], Loss: 0.1209
Epoch [500/1000], Loss: 0.1179
Epoch [600/1000], Loss: 0.1152
Epoch [700/1000], Loss: 0.1128
Epoch [800/1000], Loss: 0.1106
Epoch [900/1000], Loss: 0.1086
predictions:  tensor([1, 1, 0, 2, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 2, 1, 2, 2, 0, 0, 1, 2, 1, 2,
        2, 1, 1, 2, 2, 0, 1, 0, 0, 1, 0, 2, 2, 2, 2, 2, 0, 0, 1, 1, 1])
Y_test:  tensor([1., 1., 0., 2., 0., 1., 2., 0., 2., 0., 0., 0., 0., 0., 2., 1., 2., 2.,
        0., 0., 1., 2., 1., 2., 2., 1., 1., 2., 2., 0., 1., 0., 0., 1., 0., 2.,
        2., 2., 2., 2., 0., 0., 1., 1., 1.])
accuracy: 1.0000


In [30]:
print("SKLEARN MODEL")
model = LogisticRegression()
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(Y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

SKLEARN MODEL
Accuracy: 1.0000
